In [1]:
# !pip install praw
# !pip install pyarrow

In [2]:
import praw
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import zstandard as zstd
from datetime import datetime

### Methodology 1. Reddit praw api

In [3]:
# ### put your personal info 
# reddit = praw.Reddit(client_id='', \
#                      client_secret='', \
#                      user_agent='')

In [4]:
# posts = []
# ml_subreddit = reddit.subreddit('depression')
# for post in ml_subreddit.new(limit=1000):
#     posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
# posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])
# posts

In [5]:
# posts = []
# ml_subreddit = reddit.subreddit('depression')
# for post in ml_subreddit.hot(limit=1000):
#     posts.append([post.title, post.score, post.id, post.subreddit, post.url, post.num_comments, post.selftext, post.created])
# posts = pd.DataFrame(posts,columns=['title', 'score', 'id', 'subreddit', 'url', 'num_comments', 'body', 'created'])

In [6]:
# posts

In [7]:
# posts.title[0]

In [8]:
# posts.body[0]

### Methodology 2. Reddit archive data dump (https://the-eye.eu/redarcs/) 

In [9]:
ls

AskScienceDiscussion_submissions.zst
askscience_discussion_submissions_preprocessed.parquet
reddit_data_preprocessing_AnxietyDepression.ipynb
reddit_data_preprocessing_AskScienceDiscussion.ipynb
reddit_data_preprocessing_AutoNewspaper.ipynb
reddit_data_preprocessing_EconNews.ipynb
reddit_data_preprocessing_depressed.ipynb
reddit_data_preprocessing_technology.ipynb
reddit_data_preprocessing_webdev.ipynb
venv/
webdev_submissions.zst


In [10]:
## step1. load downloaded submission reddit dump 

In [22]:
webdev_discussion_submissions =\
    pd.read_json('webdev_submissions.zst',compression=dict(method='zstd', max_window_size=2147483648),\
    lines = True, nrows = 1000000)

In [23]:
# convert the timestamp to utc
webdev_discussion_submissions['created_utc'] =\
    webdev_discussion_submissions['created_utc'].apply(lambda x : datetime.utcfromtimestamp(x))

In [24]:
## step2. download the file into parquet file to speed up data load later stage

In [25]:
webdev_discussion_submissions.to_parquet('webdev_discussion_submissions.parquet', engine='pyarrow')

In [ ]:
## step3. read as parquet file 

In [ ]:
webdev_discussion_submissions = pd.read_parquet('webdev_discussion_submissions.parquet',engine='pyarrow')
webdev_discussion_submissions

,link_flair_css_class,created_utc,media,name,subreddit,author,gilded,downs,hide_score,stickied,quarantine,id,distinguished,selftext,media_embed,preview,subreddit_id,saved,author_flair_css_class,is_self,ups,retrieved_on,secure_media_embed,author_flair_text,archived,locked,spoiler,over_18,post_hint,title,url,link_flair_text,score,thumbnail,domain,permalink,edited,contest_mode,num_comments,secure_media,brand_safe,suggested_sort,hidden
0,NaN,2016-10-29 04:01:37,None,t3_59yt27,AutoNewspaper,AutoNewsAdmin,0,0.0,False,False,False,59yt27,moderator,,"{'content': None, 'height': None, 'scrolling':...","{'enabled': None, 'images': [{'id': 'KR8oJeB8C...",t5_3h9d4,0.0,NaN,False,2.0,1480284599,"{'content': None, 'height': None, 'scrolling':...",NaN,False,False,False,False,link,AutoNews MultiReddit,https://www.reddit.com/user/AutoNewsAdmin/m/au...,NaN,2,http://b.thumbs.redditmedia.com/Ujz94ktk0PyCHB...,reddit.com,/r/AutoNewspaper/comments/59yt27/autonews_mult...,False,False,1,None,NaN,None,NaN
1,NaN,2016-10-29 06:33:51,None,t3_59zaw2,AutoNewspaper,[deleted],0,0.0,False,False,False,59zaw2,None,[removed],"{'content': None, 'height': None, 'scrolling':...",None,t5_3h9d4,0.0,NaN,True,1.0,1480284889,"{'content': None, 'height': None, 'scrolling':...",NaN,False,False,False,False,None,[World] - 2 Indian Soldiers Killed Along Dispu...,https://www.reddit.com/r/AutoNewspaper/comment...,NaN,1,default,self.AutoNewspaper,/r/AutoNewspaper/comments/59zaw2/world_2_india...,False,False,0,None,NaN,None,NaN
2,NaN,2016-10-29 06:34:11,None,t3_59zaxg,AutoNewspaper,[deleted],0,0.0,False,False,False,59zaxg,None,[removed],"{'content': None, 'height': None, 'scrolling':...",None,t5_3h9d4,0.0,NaN,True,1.0,1480284890,"{'content': None, 'height': None, 'scrolling':...",NaN,False,False,False,False,None,[National] - 5 things to know: Transgender cas...,https://www.reddit.com/r/AutoNewspaper/comment...,NaN,1,default,self.AutoNewspaper,/r/AutoNewspaper/comments/59zaxg/national_5_th...,False,False,0,None,NaN,None,NaN
3,NaN,2016-10-29 06:34:11,None,t3_59zaxh,AutoNewspaper,[deleted],0,0.0,False,False,False,59zaxh,None,[removed],"{'content': None, 'height': None, 'scrolling':...",None,t5_3h9d4,0.0,NaN,True,1.0,1480284890,"{'content': None, 'height': None, 'scrolling':...",NaN,False,False,False,False,None,[World] - 2 Indian Soldiers Killed Along Dispu...,https://www.reddit.com/r/AutoNewspaper/comment...,NaN,1,default,self.AutoNewspaper,/r/AutoNewspaper/comments/59zaxh/world_2_india...,False,False,0,None,NaN,None,NaN
4,NaN,2016-10-29 07:10:12,None,t3_59zef2,AutoNewspaper,[deleted],0,0.0,False,False,False,59zef2,None,[removed],"{'content': None, 'height': None, 'scrolling':...",None,t5_3h9d4,0.0,NaN,True,1.0,1480284948,"{'content': None, 'height': None, 'scrolling':...",NaN,False,False,False,False,None,[Sports] - Australia survive early scare in Fa...,https://www.reddit.com/r/AutoNewspaper/comment...,NaN,1,default,self.AutoNewspaper,/r/AutoNewspaper/comments/59zef2/sports_austra...,False,False,0,None,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,NaN,2017-01-19 19:52:42,None,None,AutoNewspaper,AutoNewspaperAdmin,0,NaN,False,False,False,5oytrv,None,,"{'content': None, 'height': None, 'scrolling':...","{'enabled': False, 'images': [{'id': 's9OV58tk...",t5_3h9d4,NaN,NaN,False,NaN,1489452699,"{'content': None, 'height': None, 'scrolling':...",NaN,False,False,False,False,link,[Sports] - Faugheen and Annie Power on way bac...,https://www.theguardian.com/sport/2017/jan/19/...,NaN,1,https://b.thumbs.redditmedia.com/WFa0c_IB_LsWl...,theguardian.com,/r/AutoNewspaper/comments/5oytrv/sports_faughe...,False,False,0,None,0.0,confidence,0.0
399996,NaN,2017-01-19 19:53:01,None,None,AutoNewspaper,AutoNewspaperAdmin,0,NaN,False,False,False,5oytuh,None,,"{'content': None, 'height': None, 'scrolling':...","{'enabled': False, 'images': [{'id': 'KeRA4KOr..

In [16]:
## step4. feature selection : select only necessary columns

In [14]:
webdev_discussion_submissions =\
    webdev_discussion_submissions[['subreddit','author','over_18','link_flair_text','title','selftext','url','created_utc']]
webdev_discussion_submissions

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
0,webdev,snissn,False,None,I'll start us off with this jquery demo i set ...,,http://mseiler.com/select.php,2009-01-25 07:54:53
1,webdev,FlySwat,False,None,Web Sites != Web Applications,,http://www.gfilter.net/?Post=Web-Sites-!=-Web-...,2009-01-27 15:30:18
2,webdev,snissn,False,None,"IETester, requires windows, but allows emulati...",,http://www.my-debugbar.com/wiki/IETester/HomePage,2009-01-28 20:00:59
3,webdev,FlySwat,False,None,Handling comment spam without a captcha.,,http://gfilter.net/?Post=On-Handling-Spam,2009-02-15 05:33:20
4,webdev,FlySwat,False,None,JSManager.js - A super tiny library for dynami...,,http://www.gfilter.net/?Post=JSManager.js---Ea...,2009-02-17 05:01:30
...,...,...,...,...,...,...,...,...
354939,webdev,tedthemouse,False,None,Best Framework?,[removed],https://www.reddit.com/r/webdev/comments/1005a...,2022-12-31 22:25:15
354940,webdev,taylor-reddit,False,Question,Can anyone recommend a headless (self-hosted) ...,I’m trying to make it easy for my team and not...,https://www.reddit.com/r/webdev/comments/1006d...,2022-12-31 23:20:40
354941,webdev,808Hl,False,Question,Will someone guide me to the right steps/path ...,So far I have the domain name which I purchase...,https://www.reddit.com/r/webdev/comments/1006l...,2022-12-31 23:32:15
354942,webdev,Right_Part_5987,False,Discussion,Open-Source Projects?,"hello,\ni'm junior front end developer. i'm wi...",https://www.reddit.com/r/webdev/comments/1006w...,2022-12-31 23:49:26


In [15]:
## step5. basic pre-processing 

In [16]:
# remove blank body posts
webdev_discussion_submissions = webdev_discussion_submissions[webdev_discussion_submissions['selftext'].apply(lambda x : len(x)!=0)]
webdev_discussion_submissions

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
9,webdev,[deleted],False,None,Text Editors are soo web 1.0 :) For quick debu...,[deleted],http://pastebin.me/49ab95f66ac21,2009-03-02 08:17:30
14,webdev,[deleted],False,None,Performance Case Study: slow database queries ...,[deleted],http://2bits.com/articles/performance-case-stu...,2009-03-17 12:42:51
18,webdev,[deleted],False,None,The icecream truck analogy I just came up with...,[deleted],https://www.reddit.com/r/webdev/comments/8cykh...,2009-04-16 14:16:06
25,webdev,[deleted],False,None,Does anybody know a PHP / MySQL open source pr...,[deleted],https://www.reddit.com/r/webdev/comments/8quao...,2009-06-08 21:50:22
28,webdev,[deleted],False,None,8 Easy Steps to Slow Down your Website / Blog’...,[deleted],http://tech.inkapoint.com/website-development/...,2009-07-12 09:19:40
...,...,...,...,...,...,...,...,...
354939,webdev,tedthemouse,False,None,Best Framework?,[removed],https://www.reddit.com/r/webdev/comments/1005a...,2022-12-31 22:25:15
354940,webdev,taylor-reddit,False,Question,Can anyone recommend a headless (self-hosted) ...,I’m trying to make it easy for my team and not...,https://www.reddit.com/r/webdev/comments/1006d...,2022-12-31 23:20:40
354941,webdev,808Hl,False,Question,Will someone guide me to the right steps/path ...,So far I have the domain name which I purchase...,https://www.reddit.com/r/webdev/comments/1006l...,2022-12-31 23:32:15
354942,webdev,Right_Part_5987,False,Discussion,Open-Source Projects?,"hello,\ni'm junior front end developer. i'm wi...",https://www.reddit.com/r/webdev/comments/1006w...,2022-12-31 23:49:26


In [17]:
## opt out '[deleted]','[removed]' only selftext 
opt_out_list = ['[deleted]','[removed]']
webdev_discussion_submissions[webdev_discussion_submissions['selftext'].isin(opt_out_list)]

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
9,webdev,[deleted],False,None,Text Editors are soo web 1.0 :) For quick debu...,[deleted],http://pastebin.me/49ab95f66ac21,2009-03-02 08:17:30
14,webdev,[deleted],False,None,Performance Case Study: slow database queries ...,[deleted],http://2bits.com/articles/performance-case-stu...,2009-03-17 12:42:51
18,webdev,[deleted],False,None,The icecream truck analogy I just came up with...,[deleted],https://www.reddit.com/r/webdev/comments/8cykh...,2009-04-16 14:16:06
25,webdev,[deleted],False,None,Does anybody know a PHP / MySQL open source pr...,[deleted],https://www.reddit.com/r/webdev/comments/8quao...,2009-06-08 21:50:22
28,webdev,[deleted],False,None,8 Easy Steps to Slow Down your Website / Blog’...,[deleted],http://tech.inkapoint.com/website-development/...,2009-07-12 09:19:40
...,...,...,...,...,...,...,...,...
354929,webdev,[deleted],False,None,Happy Today's Stuff,[deleted],,2022-12-31 19:54:06
354935,webdev,[deleted],False,Question,For those who have been in Web Dev for a bit: ...,[deleted],,2022-12-31 21:29:18
354936,webdev,sweetbundie,False,None,How do I build a nested comments section?,[removed],https://www.reddit.com/r/webdev/comments/1004q...,2022-12-31 21:57:50
354938,webdev,NathanJumbo,False,Question,"Using nearly identical domain name, good or ba...",[removed],https://www.reddit.com/r/webdev/comments/10057...,2022-12-31 22:22:06


In [18]:
webdev_discussion_submissions = webdev_discussion_submissions[~webdev_discussion_submissions['selftext'].isin(opt_out_list)]
print(len(webdev_discussion_submissions))
webdev_discussion_submissions

146228


,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
37,webdev,anaranjaded,False,None,Hi WebDev- I want to make a portfolio website ...,"Hi, I am a digital artist that can't upload ar...",https://www.reddit.com/r/webdev/comments/97t84...,2009-08-05 16:50:11
44,webdev,shopireland,False,None,Need some help finding a bug in my shopping ca...,Apologies for the new account / self promotion...,https://www.reddit.com/r/webdev/comments/9k70l...,2009-09-13 23:04:52
48,webdev,illgross,False,None,What's the prevailing web technologies in use ...,Here's the deal. I promised someone to make a...,https://www.reddit.com/r/webdev/comments/9m8yk...,2009-09-20 00:01:01
59,webdev,w3bd3v,False,None,As r/webdev: What would you think of an IAmA w...,I posted this over at [reddit.com/r/iama](http...,https://www.reddit.com/r/webdev/comments/9tci5...,2009-10-12 20:09:49
69,webdev,flyface,False,None,"I'm new to web development, where should I begin?",My girlfriend recently got a promotion at her ...,https://www.reddit.com/r/webdev/comments/9wcjs...,2009-10-21 17:49:44
...,...,...,...,...,...,...,...,...
354937,webdev,gtrman571,False,Question,I think I found a bug in an npm module.,I found this npm module to create a react popu...,https://www.reddit.com/r/webdev/comments/1004x...,2022-12-31 22:07:39
354940,webdev,taylor-reddit,False,Question,Can anyone recommend a headless (self-hosted) ...,I’m trying to make it easy for my team and not...,https://www.reddit.com/r/webdev/comments/1006d...,2022-12-31 23:20:40
354941,webdev,808Hl,False,Question,Will someone guide me to the right steps/path ...,So far I have the domain name which I purchase...,https://www.reddit.com/r/webdev/comments/1006l...,2022-12-31 23:32:15
354942,webdev,Right_Part_5987,False,Discussion,Open-Source Projects?,"hello,\ni'm junior front end developer. i'm wi...",https://www.reddit.com/r/webdev/comments/1006w...,2022-12-31 23:49:26


In [19]:
# step6. final data quality check 

In [20]:
print(f'Author : {webdev_discussion_submissions.iloc[[0],:].author.tolist()[0]}')
print(f'Subreddit : {webdev_discussion_submissions.iloc[[0],:].subreddit.tolist()[0]}')
print(f'Teenager(True/False) : {webdev_discussion_submissions.iloc[[0],:].over_18.tolist()[0]}')
print(f'Posted : {webdev_discussion_submissions.iloc[[0],:].created_utc.tolist()[0]}')
print()
print(f'Title : {webdev_discussion_submissions.iloc[[0],:].title.tolist()[0]}')
print(f'Body : {webdev_discussion_submissions.iloc[[0],:].selftext.tolist()[0]}')
print()
print()
print()
print(f'Author : {webdev_discussion_submissions.iloc[[1],:].author.tolist()[0]}')
print(f'Subreddit : {webdev_discussion_submissions.iloc[[1],:].subreddit.tolist()[0]}')
print(f'Teenager(True/False) : {webdev_discussion_submissions.iloc[[1],:].over_18.tolist()[0]}')
print(f'Posted : {webdev_discussion_submissions.iloc[[1],:].created_utc.tolist()[0]}')
print()
print(f'Title : {webdev_discussion_submissions.iloc[[1],:].title.tolist()[0]}')
print(f'Body : {webdev_discussion_submissions.iloc[[1],:].selftext.tolist()[0]}')


Author : anaranjaded
Subreddit : webdev
Teenager(True/False) : False
Posted : 2009-08-05 16:50:11

Title : Hi WebDev- I want to make a portfolio website where i can easily upload and manage content. I think I should use PHP. - where should I start? /help
Body : Hi, I am a digital artist that can't upload art to my current site efficiently. I know html and css, plus minimal javascript and some coding experience, and I'm ready to overhaul my dead and dying website. I need to start from scratch and do this in php (right?). I've edited my wordpress layout but have no real php knowledge. Does anyone have any links/knowledge/starting points/advice? 

I could probably build something in wordpress since I already have it installed.

Current website is here, though mostly irrelevant because the now-empty "still art" section is the problem: 
http://kristenward.net/reelreel.html

THANK YOU!



Author : shopireland
Subreddit : webdev
Teenager(True/False) : False
Posted : 2009-09-13 23:04:52

Title

In [21]:
# step7. save final pre-processed data into parquet
webdev_discussion_submissions.to_parquet('webdev_discussion_submissions_preprocessed.parquet', engine='pyarrow')
webdev_discussion_submissions_preprocessed = pd.read_parquet('webdev_discussion_submissions_preprocessed.parquet',engine='pyarrow')
webdev_discussion_submissions_preprocessed

,subreddit,author,over_18,link_flair_text,title,selftext,url,created_utc
37,webdev,anaranjaded,False,None,Hi WebDev- I want to make a portfolio website ...,"Hi, I am a digital artist that can't upload ar...",https://www.reddit.com/r/webdev/comments/97t84...,2009-08-05 16:50:11
44,webdev,shopireland,False,None,Need some help finding a bug in my shopping ca...,Apologies for the new account / self promotion...,https://www.reddit.com/r/webdev/comments/9k70l...,2009-09-13 23:04:52
48,webdev,illgross,False,None,What's the prevailing web technologies in use ...,Here's the deal. I promised someone to make a...,https://www.reddit.com/r/webdev/comments/9m8yk...,2009-09-20 00:01:01
59,webdev,w3bd3v,False,None,As r/webdev: What would you think of an IAmA w...,I posted this over at [reddit.com/r/iama](http...,https://www.reddit.com/r/webdev/comments/9tci5...,2009-10-12 20:09:49
69,webdev,flyface,False,None,"I'm new to web development, where should I begin?",My girlfriend recently got a promotion at her ...,https://www.reddit.com/r/webdev/comments/9wcjs...,2009-10-21 17:49:44
...,...,...,...,...,...,...,...,...
354937,webdev,gtrman571,False,Question,I think I found a bug in an npm module.,I found this npm module to create a react popu...,https://www.reddit.com/r/webdev/comments/1004x...,2022-12-31 22:07:39
354940,webdev,taylor-reddit,False,Question,Can anyone recommend a headless (self-hosted) ...,I’m trying to make it easy for my team and not...,https://www.reddit.com/r/webdev/comments/1006d...,2022-12-31 23:20:40
354941,webdev,808Hl,False,Question,Will someone guide me to the right steps/path ...,So far I have the domain name which I purchase...,https://www.reddit.com/r/webdev/comments/1006l...,2022-12-31 23:32:15
354942,webdev,Right_Part_5987,False,Discussion,Open-Source Projects?,"hello,\ni'm junior front end developer. i'm wi...",https://www.reddit.com/r/webdev/comments/1006w...,2022-12-31 23:49:26
